# Criação do banco de dados

In [1]:
import sqlite3
import hashlib

In [2]:
#Conectar no Banco
conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')

In [3]:
# Criando tabela no Banco de Dados
cursor_bd = conexao_bd.cursor()

cursor_bd.execute('''CREATE TABLE LIVRO
               (ISBN TEXT PRIMARY KEY,
                URI_capa_livro TEXT,
                Autor TEXT,
                Titulo TEXT)''')

cursor_bd.execute('''CREATE TABLE MATERIAL_DIDATICO
               (ID_MATERIAL_DIDATICO INTEGER PRIMARY KEY,
                URI_foto_material TEXT,
                Numero_serie TEXT)''')

cursor_bd.execute('''CREATE TABLE RECURSO
               (ID INTEGER PRIMARY KEY,
                Data_aquisicao DATE,
                Estado_conservacao TEXT,
                Localizacao_fisica TEXT,
                Categoria TEXT,
                Descricao TEXT,
                fk_LIVRO_ISBN TEXT,
                fk_MATERIAL_DIDATICO_ID INTEGER,
                FOREIGN KEY (fk_LIVRO_ISBN) REFERENCES LIVRO (ISBN),
                FOREIGN KEY (fk_MATERIAL_DIDATICO_ID) REFERENCES MATERIAL_DIDATICO (ID_MATERIAL_DIDATICO))''')

cursor_bd.execute('''CREATE TABLE USUARIO
               (ID_USER INTEGER PRIMARY KEY,
                Nome TEXT,
                Sobrenome TEXT,
                Login TEXT,
                Senha TEXT,
                URI_foto_user TEXT,
                Funcao TEXT)''')

cursor_bd.execute('''CREATE TABLE EMPRESTIMO
               (Data_emprestimo DATE,
                Data_devolucao_prevista DATE,
                Status TEXT,
                fk_Recursos_ID_item INTEGER,
                fk_Usuarios_ID_user INTEGER,
                FOREIGN KEY (fk_Recursos_ID_item) REFERENCES RECURSO (ID),
                FOREIGN KEY (fk_Usuarios_ID_user) REFERENCES USUARIO (ID_USER),  
                PRIMARY KEY (fk_Recursos_ID_item, fk_Usuarios_ID_user))''')

# Criação da classe usuários

In [4]:
#Criando Classe Usuario
class Usuario:

    def __init__(self, nome, sobrenome, login, senha, uri_foto_user, funcao):
        self.id_user = None
        self.nome = nome
        self.sobrenome = sobrenome
        self.login = login
        self.senha = senha
        self.uri_foto_user = uri_foto_user
        self.funcao = funcao

    def __str__(self):
        return f'Nome: {self.nome} {self.sobrenome}\nLogin: {self.login}\nUri Foto Usuario: {self.uri_foto_user}\nFunção: {self.funcao}'

    def salvar(self):
        
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            salt = '5gz'
            self.senha = self.senha + salt
            hash_senha = hashlib.sha256(self.senha.encode('utf-8')).hexdigest()
            cursor.execute(f'''INSERT INTO Usuario (Nome, Sobrenome, Login, Senha, URI_foto_user, Funcao)
            VALUES ('{self.nome}', '{self.sobrenome}', '{self.login}', '{hash_senha}', '{self.uri_foto_user}', '{self.funcao}')''')
            conexao_bd.commit()
            print('Usuário cadastrado com sucesso!')
        except Exception as error:
            print('Erro ao cadastrar usuário!')
            print(error)
        finally:
            conexao_bd.close()

    def atualizar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''UPDATE Usuario SET Nome = '{self.nome}', Sobrenome = '{self.sobrenome}', Login = '{self.login}', URI_foto_user = '{self.uri_foto_user}', Funcao = '{self.funcao}' WHERE ID_USER = {self.id_user}''')
            conexao_bd.commit()
            print('Usuário atualizado com sucesso!')
        except Exception as error:
            print('Erro ao atualizar usuário!')
            print(error)
        finally:    
            conexao_bd.close()

    def deletar(self, id_user):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''DELETE FROM Usuario WHERE ID_USER = {id_user}''')
            conexao_bd.commit()
            print('Usuário deletado com sucesso!')
        except Exception as error:
            print('Erro ao deletar usuário!')
            print(error)
        finally:
            conexao_bd.close()

    def buscar(login):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''SELECT * FROM Usuario WHERE Login = "{login}"''')
            resultados = cursor.fetchall()
            linha = resultados[0]
            id_user = linha[0]
            nome = linha[1]
            sobrenome = linha[2]
            login = linha[3]
            senha = linha[4]
            uri_foto_user = linha[5]
            funcao = linha[6]
            usuario = Usuario(nome, sobrenome, login, senha, uri_foto_user, funcao)
            usuario.id_user = id_user
            return usuario
        
        except Exception as error:
            print('Erro ao buscar usuário!')
            print(error)
        finally:
            conexao_bd.close()
            

    def set_funcao(self, funcao):
        if funcao == 'Administrador' or funcao == 'Chefe do laboratório' or funcao == 'Membro do laboratorio':
            self.funcao = funcao
        else:  
            print('Função inválida!')
            throw('Função inválida!')

    def get_funcao(self):
        return self.funcao
    



# Testes da classe usuários

In [5]:
#Criar usuário novo
usuario1 = Usuario('João', 'Silva', 'joao.silva', '123456', 'https://www.google.com.br', 'Administrador')

In [6]:
#salvar usuário no banco
usuario1.salvar()

Usuário cadastrado com sucesso!


In [7]:
#atualizar usuário no banco
usuario1 = Usuario.buscar('joao.silva')
usuario1.nome = 'Tião'
usuario1.atualizar()

Usuário atualizado com sucesso!


In [8]:
#buscar usuário no banco
usuarioPesquisado = Usuario.buscar('joao.silva')
print(usuarioPesquisado)

Nome: Tião Silva
Login: joao.silva
Uri Foto Usuario: https://www.google.com.br
Função: Administrador


# Criação da classe recursos

In [9]:
#Criar classe Recurso
class Recurso:

    def __init__(self, data_aquisicao, estado_conservacao, localizacao_fisica, categoria, descricao, fk_livro_isbn, fk_material_didatico_id):
        self.id_recurso = None
        self.data_aquisicao = data_aquisicao
        self.estado_conservacao = estado_conservacao
        self.localizacao_fisica = localizacao_fisica
        self.categoria = categoria
        self.descricao = descricao
        self.fk_livro_isbn = fk_livro_isbn
        self.fk_material_didatico_id = fk_material_didatico_id

    def __str__(self):
        return f'Descrição: {self.descricao}\nCategoria: {self.categoria}\nData de Aquisição: {self.data_aquisicao}\nEstado de Conservação: {self.estado_conservacao}\nLocalização Física: {self.localizacao_fisica}\nID Material Didático: {self.fk_material_didatico_id}\nISBN Livro: {self.fk_livro_isbn}'

    def salvar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''INSERT INTO Recurso (Data_Aquisicao, Estado_Conservacao, Localizacao_Fisica, Categoria, Descricao, FK_Livro_ISBN, FK_Material_Didatico_ID)
            VALUES ('{self.data_aquisicao}', '{self.estado_conservacao}', '{self.localizacao_fisica}', '{self.categoria}', '{self.descricao}', '{self.fk_livro_isbn}', '{self.fk_material_didatico_id}')''')
            conexao_bd.commit()
            print('Recurso cadastrado com sucesso!')
        except Exception as error:
            print('Erro ao cadastrar recurso!')
            print(error)
        finally:
            conexao_bd.close()

    def atualizar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''UPDATE Recurso SET Data_Aquisicao = '{self.data_aquisicao}', Estado_Conservacao = '{self.estado_conservacao}', Localizacao_Fisica = '{self.localizacao_fisica}', Categoria = '{self.categoria}', Descricao = '{self.descricao}', FK_Livro_ISBN = '{self.fk_livro_isbn}', FK_Material_Didatico_ID = '{self.fk_material_didatico_id}' WHERE ID = {self.id_recurso}''')
            conexao_bd.commit()
            print('Recurso atualizado com sucesso!')
        except Exception as error:
            print('Erro ao atualizar recurso!')
            print(error)
        finally:    
            conexao_bd.close()

    def deletar(self, id_recurso):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''DELETE FROM Recurso WHERE ID = {id_recurso}''')
            conexao_bd.commit()
            print('Recurso deletado com sucesso!')
        except Exception as error:
            print('Erro ao deletar recurso!')
            print(error)
        finally:
            conexao_bd.close()

    def buscar(fk_material_didatico_id):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''SELECT * FROM Recurso WHERE fk_MATERIAL_DIDATICO_id = "{fk_material_didatico_id}"''')
            resultados = cursor.fetchall()
            linha = resultados[0]
            id_recurso = linha[0]
            data_aquisicao = linha[1]
            estado_conservacao = linha[2]
            localizacao_fisica = linha[3]
            categoria = linha[4]
            descricao = linha[5]
            fk_livro_isbn = linha[6]
            fk_material_didatico_id = linha[7]
            recurso = Recurso(data_aquisicao, estado_conservacao, localizacao_fisica, categoria, descricao, fk_livro_isbn, fk_material_didatico_id)
            recurso.id_recurso = id_recurso
            return recurso
        
        except Exception as error:
            print('Erro ao buscar recurso!')
            print(error)
        finally:
            conexao_bd.close()

# Testes da classe recursos

In [10]:
#Criar recurso novo
recurso1 = Recurso("2023-11-02","Bom","E1P1","Saúde","livro doido","","0123")

In [11]:
#salvar recurso no banco
recurso1.salvar()

Recurso cadastrado com sucesso!


In [12]:
# atualizar recurso no banco
recurso1 = Recurso.buscar('0123')
recurso1.categoria = 'Matemática'
recurso1.atualizar()

Recurso atualizado com sucesso!


In [13]:
#buscar recurso no banco
recursoPesquisado = Recurso.buscar('0123')
print(recursoPesquisado)

Descrição: livro doido
Categoria: Matemática
Data de Aquisição: 2023-11-02
Estado de Conservação: Bom
Localização Física: E1P1
ID Material Didático: 123
ISBN Livro: 


# Criação da classe Material Didático

In [14]:
import sqlite3

class MaterialDidatico:

    def __init__(self, uri_foto_material, numero_serie):
        self.id_material_didatico = None
        self.uri_foto_material = uri_foto_material
        self.numero_serie = numero_serie

    def __str__(self):
        return f'URI Foto Material: {self.uri_foto_material}\nNúmero de Série: {self.numero_serie}'

    def salvar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''INSERT INTO Material_Didatico (URI_foto_material, Numero_serie)
            VALUES ('{self.uri_foto_material}', '{self.numero_serie}')''')
            conexao_bd.commit()
            print('Material Didático cadastrado com sucesso!')
        except Exception as error:
            print('Erro ao cadastrar Material Didático!')
            print(error)
        finally:
            conexao_bd.close()

    def atualizar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''UPDATE Material_Didatico SET URI_foto_material = '{self.uri_foto_material}', Numero_serie = '{self.numero_serie}' WHERE ID_MATERIAL_DIDATICO = {self.id_material_didatico}''')
            conexao_bd.commit()
            print('Material Didático atualizado com sucesso!')
        except Exception as error:
            print('Erro ao atualizar Material Didático!')
            print(error)
        finally:    
            conexao_bd.close()

    def deletar(self, id_material_didatico):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''DELETE FROM Material_Didatico WHERE ID_MATERIAL_DIDATICO = {id_material_didatico}''')
            conexao_bd.commit()
            print('Material Didático deletado com sucesso!')
        except Exception as error:
            print('Erro ao deletar Material Didático!')
            print(error)
        finally:
            conexao_bd.close()

    def buscar(numero_serie):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''SELECT * FROM Material_Didatico WHERE Numero_serie = "{numero_serie}"''')
            resultados = cursor.fetchall()
            linha = resultados[0]
            id_material_didatico = linha[0]
            uri_foto_material = linha[1]
            numero_serie = linha[2]
            material_didatico = MaterialDidatico(uri_foto_material, numero_serie)
            material_didatico.id_material_didatico = id_material_didatico
            return material_didatico
        
        except Exception as error:
            print('Erro ao buscar Material Didático!')
            print(error)
        finally:
            conexao_bd.close()


# Testes da classe de materiais didáticos

In [15]:
#Criar material didático novo
material1 = MaterialDidatico("", "MD2023-876543")

In [16]:
#salvar material didático no banco
material1.salvar()

Material Didático cadastrado com sucesso!


In [17]:
# atualizar material didático no banco
material1 = MaterialDidatico.buscar('MD2023-876543')
material1.uri_foto_material = 'www.com'
material1.atualizar()

Material Didático atualizado com sucesso!


In [18]:
#buscar material didático no banco
materialPesquisado = MaterialDidatico.buscar('MD2023-876543')
print(materialPesquisado)

URI Foto Material: www.com
Número de Série: MD2023-876543


# Criação da classe de Livros

In [19]:
class Livros:

    def __init__(self, isbn, autor, titulo, uri_capa_livro):
        self.titulo = titulo
        self.isbn = isbn
        self.autor = autor # Devemos separar nome e sobrenome?
        self.uri_capa_livro = uri_capa_livro

    def __str__(self):
        return f'Título: {self.titulo}\nISBN: {self.isbn}\nAutor: {self.autor}\nUri Foto Usuario: {self.uri_capa_livro}\n'

    def salvar(self):
        
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''INSERT INTO Livro (Titulo, ISBN, Autor, URI_capa_livro)
            VALUES ('{self.titulo}', '{self.isbn}', '{self.autor}', '{self.uri_capa_livro}')''')
            conexao_bd.commit()
            print('Livro cadastrado com sucesso!')
        except Exception as error:
            print('Erro ao cadastrar livro!')
            print(error)
        finally:
            conexao_bd.close()
            
    def atualizar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''UPDATE Livro SET Titulo = '{self.titulo}', ISBN = '{self.isbn}', Autor = '{self.autor}', URI_capa_livro = '{self.uri_capa_livro}' WHERE ISBN = {self.isbn}''')
            conexao_bd.commit()
            print('Livro atualizado com sucesso!') # colocar mais detalhes sobre o livro
        except Exception as error:
            print('Erro ao atualizar livro!')
            print(error)
        finally:    
            conexao_bd.close()

    def deletar(self, isbn): # pode acontecer um problema em deletar todos com mesmo ISBN
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''DELETE FROM Livro WHERE ISBN = {isbn}''')
            conexao_bd.commit()
            print('Livro deletado com sucesso!')
        except Exception as error:
            print('Erro ao deletar o livro!')
            print(error)
        finally:
            conexao_bd.close()

    def buscar(isbn):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''SELECT * FROM Livro WHERE ISBN = "{isbn}"''')
            resultados = cursor.fetchall()
            linha = resultados[0]
            cod_isbn = linha[0]
            uri_capa_livro = linha[1]
            autor = linha[2]
            titulo = linha[3]
            livro = Livros(cod_isbn, autor, titulo, uri_capa_livro)
            livro.isbn = cod_isbn
            return livro
        
        except Exception as error:
            print('Erro ao buscar o livro!')
            print(error)
        finally:
            conexao_bd.close()
            

# Testes da classe de Livros

In [20]:
#Criar livro novo
livro1 = Livros('8506086892', 'José Mauro', 'Meu Pé de Laranja Lima', 'https://www.google.com.br')

In [21]:
#salvar livro no banco
livro1.salvar()
print(livro1)

Livro cadastrado com sucesso!
Título: Meu Pé de Laranja Lima
ISBN: 8506086892
Autor: José Mauro
Uri Foto Usuario: https://www.google.com.br



In [22]:
#atualizar livro no banco
livro1 = Livros.buscar('8506086892')
livro1.titulo = 'Meu Pé de limão'
livro1.atualizar()

Livro atualizado com sucesso!


In [23]:
#buscar livro no banco
livro_pesquisado = Livros.buscar('8506086892')
print(livro_pesquisado)

Título: Meu Pé de limão
ISBN: 8506086892
Autor: José Mauro
Uri Foto Usuario: https://www.google.com.br



# Criação da classe empréstimos

In [24]:
class Emprestimos:

    def __init__(self, data_emprestimo, data_devolucao_prevista, status, id_recurso, id_usuario):
        self.id_emprestimo = None
        self.data_emprestimo = data_emprestimo
        self.data_devolucao_prevista = data_devolucao_prevista
        self.status = status
        self.id_recurso = id_recurso
        self.id_usuario = id_usuario

    def __str__(self):
        return f'ID do usuário: {self.id_usuario}\nID do recurso: {self.id_recurso}\nStatus: {self.status}\nData do empréstimo: {self.data_emprestimo}\nData prevista para devolução: {self.data_devolucao_prevista}'

    def salvar(self):
        
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''INSERT INTO Emprestimo (Data_emprestimo, Data_devolucao_prevista, Status, fk_Recursos_ID_item, fk_Usuarios_ID_user)
            VALUES ('{self.data_emprestimo}', '{self.data_devolucao_prevista}', '{self.status}', '{self.id_recurso}', '{self.id_usuario}')''')
            conexao_bd.commit()
            print('Empréstimo realizado com sucesso!')
        except Exception as error:
            print('Erro ao realizar empréstimo!')
            print(error)
        finally:
            conexao_bd.close()
            
    def atualizar(self):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''UPDATE Emprestimo SET Data_emprestimo = '{self.data_emprestimo}',
                                                      Data_devolucao_prevista = '{self.data_devolucao_prevista}',
                                                      Status = '{self.status}',
                                                      fk_Recursos_ID_item = {self.id_recurso},
                                                      fk_Usuarios_ID_user = {self.id_usuario}
                              WHERE fk_Recursos_ID_item = {self.id_recurso} AND fk_Usuarios_ID_user = {self.id_usuario}''')
            conexao_bd.commit()
            print('Emprestimo atualizado com sucesso!')
        except Exception as error:
            print('Erro ao atualizar emprestimo!')
            print(error)
        finally:
            conexao_bd.close()

    def deletar(self, id_recurso, id_usuario): # pode acontecer um problema em deletar todos com mesmo ISBN
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''DELETE FROM Emprestimo WHERE fk_Recursos_ID_item= {id_recurso} AND fk_Usuarios_ID_user={id_usuario}''')
            conexao_bd.commit()
            print('Emprestimo deletado com sucesso!')
        except Exception as error:
            print('Erro ao deletar o emprestimo!')
            print(error)
        finally:
            conexao_bd.close()

    def buscar(id_recurso, id_usuario):
        try:
            conexao_bd = sqlite3.connect('sgmld_sem_heranca_alim.db')
            cursor = conexao_bd.cursor()
            cursor.execute(f'''SELECT * FROM Emprestimo WHERE fk_Recursos_ID_item = {id_recurso} AND fk_Usuarios_ID_user = {id_usuario}''')
            resultados = cursor.fetchall()
            linha = resultados[0]
            data_emprestimo = linha[0]
            data_devolucao_prevista = linha[1]
            status = linha[2]
            id_recurso = linha[3]
            id_usuario = linha[4]
            emprestimo = Emprestimos(data_emprestimo, data_devolucao_prevista, status, id_recurso, id_usuario)
            #emprestimo.id = id_recurso and id_usuario
            return emprestimo
        
        except Exception as error:
            print('Erro ao buscar o livro!')
            print(error)
        finally:
            conexao_bd.close()
            

# Teste da classe Empréstimos

In [25]:

emprestimo1 = Emprestimos('01/01/2000', '01/01/2000', 'Concluído', recurso1.id_recurso, usuario1.id_user)

In [26]:
#salvar emprestimo no banco
emprestimo1.salvar()
print(emprestimo1)

Empréstimo realizado com sucesso!
ID do usuário: 1
ID do recurso: 1
Status: Concluído
Data do empréstimo: 01/01/2000
Data prevista para devolução: 01/01/2000


In [27]:
#atualizar emprestimo no banco
emprestimo1 = Emprestimos.buscar(recurso1.id_recurso, usuario1.id_user)
emprestimo1.status = 'Em andamento'
emprestimo1.atualizar()

Emprestimo atualizado com sucesso!


In [28]:
#buscar emprestimo no banco
emprestimo_pesquisado = Emprestimos.buscar(recurso1.id_recurso, usuario1.id_user)
print(emprestimo_pesquisado)

ID do usuário: 1
ID do recurso: 1
Status: Em andamento
Data do empréstimo: 01/01/2000
Data prevista para devolução: 01/01/2000


## TO-DO
- O banco deve ser criado ao iniciar pela primeira vez. Ao executar todas as vezes, verifique se o banco existe, se existir prossiga com o código, se nao existir, crie um novo banco e prossiga.
- Aumentar quantidade de testes.
- Aumentar quantidade de comentários no código.